Data Preparation (Pandas + NLTK )   
Clean and Normalize Data (Local with Pandas + NLTK)

In [1]:

import pandas as pd
df1 = pd.read_csv("nike-data-1.csv")
df2 = pd. read_csv("nike-data-2.csv")
if 'uniq_id' in df2.columns:
    df2 = df2.drop(columns=['uniq_id'])
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

# Concatenate side-by-side
combined_df = pd.concat([df2, df1], axis=1)

# Create new sequential id column
combined_df.insert(0, 'new_id', range(1, len(combined_df) + 1))

combined_df.head(5)


,new_id,Title,Subtitle,Product Description,url,name,sub_title,brand,model,color,...,currency,availability,description,raw_description,avg_rating,review_count,images,available_sizes,uniq_id,scraped_at
0,1,Nike Air Force 1 '07,Men's Shoes,It doesn't get more legendary than this. Desig...,https://www.nike.com/t/dri-fit-team-minnesota-...,Nike Dri-FIT Team (MLB Minnesota Twins),Men's Long-Sleeve T-Shirt,Nike,14226571.0,Navy,...,USD,InStock,SWEAT-WICKING COMFORT.The Nike Dri-FIT Team (M...,"<div class=""pi-pdpmainbody""><p><b class=""headl...",NaN,NaN,https://static.nike.com/a/images/t_PDP_1280_v1...,S | M | L | XL | 2XL,c3229e54-aa58-5fdd-9f71-fbe66366b2b2,20/09/2022 23:32:28
1,2,Nike Air Max Dawn SE,Men's Shoes,Find out what moves you with the Air Max Dawn....,https://www.nike.com/t/club-américa-womens-dri...,Club América,Women's Nike Dri-FIT Soccer Jersey Dress,Nike,13814665.0,Black/Black,...,USD,InStock,"Inspired by traditional soccer jerseys, the Cl...","<div class=""pi-pdpmainbody""><br/><p>Inspired b...",5.0,1.0,https://static.nike.com/a/images/t_PDP_1280_v1...,L (12–14),f8ebb2ed-17ae-5719-b750-5ea3ec69b75c,20/09/2022 23:32:40
2,3,Nike SB Dunk Low Pro Premium,Skate Shoes,Pack your style—on your feet. Bringing a fresh...,https://www.nike.com/t/sportswear-swoosh-mens-...,Nike Sportswear Swoosh,Men's Overalls,Nike,13015648.0,Black/White,...,USD,OutOfStock,WORKING HARD TO KEEP YOU COMFORTABLE.The Nike ...,"<div class=""pi-pdpmainbody""><p><b class=""headl...",4.9,11.0,https://static.nike.com/a/images/t_PDP_1280_v1...,NaN,88120081-e6cb-5399-b9dc-a2d3d5dd5206,20/09/2022 23:33:16
3,4,Nike Air Force 1 Mid '07 LX,Men's Shoes,The celebrations just keep coming. Unbox the A...,https://www.nike.com/t/dri-fit-one-luxe-big-ki...,Nike Dri-FIT One Luxe,Big Kids' (Girls') Printed Tights (Extended Size),Nike,13809796.0,Black/Rush Pink,...,USD,OutOfStock,ELEVATED COMFORT GOES FULL BLOOM.The Nike Dri-...,"<div class=""pi-pdpmainbody""><p><b class=""headl...",NaN,NaN,https://static.nike.com/a/images/t_PDP_1280_v1...,NaN,98348cc5-1520-5b6e-a5f6-c42547b6a092,20/09/2022 23:33:17
4,5,Nike Air Force 1 Mid '07,Men's Shoes,"Got your fave colour yet? No worries, the Colo...",https://www.nike.com/t/paris-saint-germain-rep...,Paris Saint-Germain Repel Academy AWF,Big Kids' Soccer Jacket,Nike,13327415.0,Dark Grey/Black/Siren Red/Siren Red,...,USD,InStock,WATER-REPELLENT COVERAGE GETS PSG DETAILS.The ...,"<div class=""pi-pdpmainbody""><p><b class=""headl...",NaN,NaN,https://static.nike.com/a/images/t_PDP_1280_v1...,XS | S | M | L | XL,f15981a5-d8c9-53fa-880d-80606be188fe,20/09/2022 23:33:22


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', str(text).lower())
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    tokens = [WordNetLemmatizer().lemmatize(t) for t in tokens]
    return ' '.join(tokens)

combined_df['processed_title'] = combined_df['Title'].apply(preprocess_text)
combined_df['processed_description'] = combined_df['Product Description'].apply(preprocess_text)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
!pip install pyspark

from pyspark.sql import SparkSession

# Safe initialization: only creates a new session if it doesn't already exist
if 'spark' not in locals():
    spark = SparkSession.builder.master("local[*]").appName("SearchPreprocessing").getOrCreate()



df_spark = spark.createDataFrame(combined_df)
df_spark.printSchema()



root
 |-- new_id: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- Product Description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- sub_title: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: double (nullable = true)
 |-- color: string (nullable = true)
 |-- price: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- availability: string (nullable = true)
 |-- description: string (nullable = true)
 |-- raw_description: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- review_count: double (nullable = true)
 |-- images: string (nullable = true)
 |-- available_sizes: string (nullable = true)
 |-- uniq_id: string (nullable = true)
 |-- scraped_at: string (nullable = true)
 |-- processed_title: string (nullable = true)
 |-- processed_description: string (nullable = true)



EDA (Exploratory Data Analysis)

In [5]:
print("Total products:", len(combined_df))
print("Available colors:", combined_df['color'].nunique())
print(combined_df['availability'].value_counts())


Total products: 400
Available colors: 76
availability
InStock       67
OutOfStock    41
Name: count, dtype: int64


Query Expansion with WordNet (Semantic Search)

In [6]:
from nltk.corpus import wordnet

def expand_query(query):
    words = word_tokenize(query.lower())
    synonyms = set(words)
    for word in words:
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)

 Feature Engineering for XGBoost

In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

combined_df['text'] = combined_df['name'].fillna('') + ' ' + combined_df['sub_title'].fillna('') + ' ' + combined_df['processed_description']
combined_df['embedding'] = combined_df['text'].apply(lambda x: model.encode(x))


Expand Query, Compute Similarities

In [13]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Expand query with synonyms
def expand_query(query):
    words = word_tokenize(query.lower())
    synonyms = set(words)
    for word in words:
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)

# Semantic search
def get_relevant_products(query, top_k=10):
    query_embedding = model.encode(query)
    combined_df['semantic_sim'] = combined_df['embedding'].apply(lambda emb: util.cos_sim(query_embedding, emb).item())
    results = combined_df.sort_values('semantic_sim', ascending=False).head(top_k)
    return results[['name', 'semantic_sim', 'processed_title', 'processed_description']]


Keyword Match Score (basic feature)

In [16]:
def keyword_match(query, text):
    query_words = set(str(query).lower().split())
    text_words = set(str(text).lower().split())
    return len(query_words & text_words) / len(query_words) if query_words else 0.0


combined_df['keyword_match_score'] = combined_df.apply(
    lambda row: keyword_match(row['sub_title'], row['processed_description']),
    axis=1
)



Generate Relevance Labels (Simulated)

In [17]:
query = "Men's Running Shoes"
query_embedding = model.encode(query)

# Compute semantic similarity
combined_df['semantic_sim'] = combined_df['embedding'].apply(lambda emb: util.cos_sim(query_embedding, emb).item())

# Sort and assign simulated relevance (top 10 → 3, mid → 2, rest → 1)
combined_df['relevance'] = 1  # default low
combined_df.loc[combined_df['semantic_sim'] > 0.6, 'relevance'] = 3
combined_df.loc[(combined_df['semantic_sim'] > 0.4) & (combined_df['semantic_sim'] <= 0.6), 'relevance'] = 2


XGBoost Relevance Model

In [24]:
from xgboost import XGBRanker
from sklearn.model_selection import train_test_split
from math import ceil

features = ['keyword_match_score', 'semantic_sim']
X = combined_df[features]
y = combined_df['relevance']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

n_train = len(X_train)
group_size = 10
num_full_groups = n_train // group_size
last_group_size = n_train % group_size

train_group = [group_size] * num_full_groups
if last_group_size > 0:
    train_group.append(last_group_size)

assert sum(train_group) == n_train

# Initialize model BEFORE fitting
model_xgb = XGBRanker(objective='rank:pairwise', n_estimators=100)

# Fit the model using the correct train group sizes
model_xgb.fit(X_train, y_train, group=train_group)



XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None,
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=100,
          n_jobs=None, num_parallel_tree=None, objective='rank:pairwise', ...)

Ranking Metrics   Evaluation Metrics (Precision@K, Recall@K, MRR, NDCG)

In [35]:
import numpy as np
from sklearn.metrics import ndcg_score

def precision_at_k(y_true, y_pred, k=10, rel_threshold=1):
    order = np.argsort(y_pred)[::-1][:k]
    y_true_k = y_true[order]
    return np.sum(y_true_k > rel_threshold) / k

def recall_at_k(y_true, y_pred, k=10, rel_threshold=1):
    order = np.argsort(y_pred)[::-1][:k]
    y_true_k = y_true[order]
    num_relevant = np.sum(y_true > rel_threshold)
    if num_relevant == 0:
        return 0.0
    return np.sum(y_true_k > rel_threshold) / num_relevant

def mean_reciprocal_rank(y_true, y_pred, rel_threshold=1):
    order = np.argsort(y_pred)[::-1]
    y_true_ordered = y_true[order]
    for i, rel in enumerate(y_true_ordered, start=1):
        if rel > rel_threshold:
            return 1.0 / i
    return 0.0

def evaluate_group(y_true, y_pred, k=10, rel_threshold=1):
    p = precision_at_k(y_true, y_pred, k, rel_threshold)
    r = recall_at_k(y_true, y_pred, k, rel_threshold)
    mrr = mean_reciprocal_rank(y_true, y_pred, rel_threshold)
    ndcg = ndcg_score([y_true], [y_pred], k=k)
    return p, r, mrr, ndcg

# group_test defines the size of each query group in the test set
group_test = [10] * (len(X_test) // 10)  # Example, adjust as needed

y_pred = model_xgb.predict(X_test)
y_true = y_test.values

start = 0
for idx, group_size in enumerate(group_test):
    end = start + group_size
    y_true_group = y_true[start:end]
    y_pred_group = y_pred[start:end]

    p, r, mrr, ndcg = evaluate_group(y_true_group, y_pred_group, k=10, rel_threshold=1)

    print(f"Group {idx + 1} Metrics:")
    print(f"  Precision@10: {p:.4f}")
    print(f"  Recall@10: {r:.4f}")
    print(f"  MRR: {mrr:.4f}")
    print(f"  NDCG@10: {ndcg:.4f}\n")

    start = end
print("Example y_true and y_pred from group 1:")
print("y_true:", y_true[0:10])
print("y_pred:", y_pred[0:10])




Group 1 Metrics:
  Precision@10: 0.2000
  Recall@10: 1.0000
  MRR: 1.0000
  NDCG@10: 1.0000

Group 2 Metrics:
  Precision@10: 0.3000
  Recall@10: 1.0000
  MRR: 1.0000
  NDCG@10: 1.0000

Group 3 Metrics:
  Precision@10: 0.1000
  Recall@10: 1.0000
  MRR: 1.0000
  NDCG@10: 1.0000

Group 4 Metrics:
  Precision@10: 0.3000
  Recall@10: 1.0000
  MRR: 1.0000
  NDCG@10: 1.0000

Group 5 Metrics:
  Precision@10: 0.1000
  Recall@10: 1.0000
  MRR: 1.0000
  NDCG@10: 1.0000

Group 6 Metrics:
  Precision@10: 0.1000
  Recall@10: 1.0000
  MRR: 1.0000
  NDCG@10: 1.0000

Group 7 Metrics:
  Precision@10: 0.0000
  Recall@10: 0.0000
  MRR: 0.0000
  NDCG@10: 1.0000

Group 8 Metrics:
  Precision@10: 0.2000
  Recall@10: 1.0000
  MRR: 1.0000
  NDCG@10: 1.0000

Example y_true and y_pred from group 1:
y_true: [1 1 1 1 1 2 1 1 1 2]
y_pred: [-2.7506099 -2.7506099 -2.7506099 -2.7506099 -2.7506099  2.7343066
 -2.7506099 -2.7506099 -2.7506099  2.7343066]
